In [2]:
from pathlib import Path
from ultralytics import YOLO

## 1. P&ID Pre-Processing
    - 7,168 W x 4,561 H Un-Cropped
    - 5,668 W x 4,311 H Cropped (Removed Title Block & Border)

In [2]:
from PIDObjectDetector import PIDPreprocessor

### 1. Crop Image & Adjust Annotations

In [3]:
root = Path().resolve().parents[1] # Data folder is 1 dir up

In [4]:
WIDTH = 7168 # pixels
HEIGHT = 4561 # pixels

# New Boundaries after Crop
X1_CROP = 400
Y1_CROP = 300
X2_CROP = 5668 
Y2_CROP = 4311

pids = PIDPreprocessor(
        root,
        WIDTH,
        HEIGHT,
        475,
        400,
        general_classes=False,
        crop_coords= (X1_CROP, Y1_CROP, X2_CROP, Y2_CROP)
)

# Crop Images & Adjust Annotations 
pids.yolo_pre_processing() 

TypeError: PIDPreprocessor.yolo_pre_processing() missing 1 required positional argument: 'folder'

### 2. Segment Cropped Images

#### - Segment training/val data to different tile sizes so that the aspect ratio of the symbol to the tile is varied in the training data. Modify overlap ratios so that the stride length is same between tiles (same number of training images for each tile size)

1. 320 px tile
    - 320 pixel stride
        - 0% overlap ratio
2. 480 px tile
    - 480 pixel stride
        - 33% overlap ratio
2. 640 px tile
    - 480 pixel stride
        - 50% overlap ratio 
3. 800 px tile
    - 480 pixel stride 
        - 67% overlap ratio

In [13]:
# New Dimensions of Cropped Image
WIDTH = 5268
HEIGHT = 4011

cropped_pids = PIDPreprocessor(
        root,
        WIDTH,
        HEIGHT,
        475,
        400,
        general_classes=False,
)

In [15]:
cropped_pids.segment_training_imgs('synthetic_1_cropped', tile_size=320, overlap_ratio=0)

In [7]:
cropped_pids.segment_training_imgs('synthetic_1_cropped', tile_size=480, overlap_ratio=0.33)

In [8]:
cropped_pids.segment_training_imgs('synthetic_1_cropped', tile_size=640, overlap_ratio=0.5)

In [9]:
cropped_pids.segment_training_imgs('synthetic_1_cropped', tile_size=800, overlap_ratio=0.67)

## 2. YOLO Training

### YOLO Finetuned on P&ID Images. 

#### Classes:
    1 'Gate Valve, N.C',
    2 'Ball Valve', 
    3 'Globe Valve, N.O.', 
    4 'Gate Valve, N.O.', 
    5 'Ball Valve', 
    6 'Butterfly Valve', 
    7 'Gate Valve, Manually Actuated', 
    8 'Check Valve', 
    9 'Diaphragm Valve', 
    10 'Needle Valve', 
    11 'Gate Valve, P.C.', 
    12 'Gate Valve, N.C.', 
    13 'Globe Valve, N.C',
    14 'Gate Valve, Pneumatic', 
    15 'Rotary Valve, N.O.', 
    16 'Rotary Valve, N.C.', 
    17 'Open Spacer', 
    18 'Spectacle Blind, N.C.', 
    19 'Spectacle Blind, N.O.', 
    20 'Reducer', 
    21 'Flange', 
    22 'Heat Exchanger',
    23 'Insulation', 
    24 'Flow Arrow', 
    25 'Circle Valve',
    26 'Inline Indicator', 
    27 'Inline Flow Meter',
    28 'Shared Display, Local',
    29 'Shared Display, Remote',
    30 'Control Panel',
    31 'Non-Inline Flow Meter',
    32 'Control Measurement Point',
    33 'Gauge',
    34 'PLC',
    35 'Swinging Disk Check Valve',
    36 'Hose Bib',
    37 'Pressure Vessel',
    38 'Centrifugal Pump',
    39 'Motor',
    40 'Coupling',
    41 'Continuation Arrow'

In [ ]:
# Pre-Trained Yolo v 11
model = YOLO('yolo11n.pt')

model.train(
        data = 'YAML/data_cropped_segmented.yaml',
        epochs = 100,
        patience = 10,
        batch = 32,
        imgsz = 640, # og image size 7,168, cropped 5,668
        name = 'Single Class 100E (MR Slices, 90 deg)',
        device = 0,
        verbose = True,
        seed = 510,
        fraction = 0.3, 
        cos_lr = True,
        single_cls = True,
        dropout = 0.3,
        plots = True,

        # Augmentation Parameters #
        hsv_h = 0, # P&IDs will be b/w, don't modify hue
        hsv_s = 0, # or saturation
        hsv_v = 0.3, # Brightness Augmentation (counterract gray mean borders introduced by rotation/shearing augms)
        degrees = 90,
        translate = 0,
        scale = 0.2, 
        shear = 10,
        flipud = 0.5,
        fliplr = 0.5, 
        mosaic = 0, # Disable mosaic, larger P&ID crops already mosaic like
        mixup = 0,  
        erasing = 0.3, # Erases portion of images
        crop_fraction = 1.0, # Disable cropping, reduce gray borders during training
)

New https://pypi.org/project/ultralytics/8.3.29 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.6  Python-3.11.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=YAML/data_cropped_segmented.yaml, epochs=100, time=None, patience=10, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=Single Class 100E (MR Slices), exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=510, deterministic=True, single_cls=True, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=0.3, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fals

train: Scanning C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\synthetic_1_cropped_segmented_multi_resolution\labels\train... 78992 images, 24328 backgrounds, 0 corrupt: 100%|██████████| 103320/103320 [03:07<00:00, 551.42it/s] 


train: New cache created: C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\synthetic_1_cropped_segmented_multi_resolution\labels\train.cache


val: Scanning C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\synthetic_1_cropped_segmented_multi_resolution\labels\val... 48132 images, 16443 backgrounds, 0 corrupt: 100%|██████████| 64575/64575 [00:40<00:00, 1579.59it/s]


val: New cache created: C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\synthetic_1_cropped_segmented_multi_resolution\labels\val.cache
Plotting labels to runs\detect\Single Class 100E (MR Slices)\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\Single Class 100E (MR Slices)
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      4.88G     0.2211     0.1273     0.7928         53        640: 100%|██████████| 3229/3229 [08:13<00:00,  6.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1009/1009 [04:06<00:00,  4.09it/s]


                   all      64575     138978      0.992      0.993      0.995      0.986

100 epochs completed in 21.819 hours.
Optimizer stripped from runs\detect\Single Class 100E (MR Slices)\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\Single Class 100E (MR Slices)\weights\best.pt, 5.5MB

Validating runs\detect\Single Class 100E (MR Slices)\weights\best.pt...
Ultralytics 8.3.6  Python-3.11.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
YOLO11n summary (fused): 238 layers, 2,589,952 parameters, 0 gradients, 6.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1009/1009 [04:39<00:00,  3.61it/s]


                   all      64575     138978      0.992      0.993      0.995      0.986
       Gate Valve, N.O      48132     138978      0.992      0.993      0.995      0.986
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\Single Class 100E (MR Slices)


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002015EF10550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480